In [1]:
%load_ext Cython
import numpy as np
np.set_printoptions(precision=2,suppress=True,linewidth=250,threshold=2000)

In [2]:
import numpy as np
import pandas as pd
import pyBigWig
import math
import csv
import multiprocessing

In [3]:
bw = pyBigWig.open("/home/musab/bigwig/wgEncodeSydhTfbsHepg2Arid3anb100279IggrabSig.bigWig")
chromo = 'chr14'
total_size = bw.chroms()[chromo]
F=bw.values(chromo,0,total_size,numpy=True)
F[np.isnan(F)] = 0

In [4]:
%%cython -a

import numpy as np
import pandas as pd
import math
import copy
import os
import cython

@cython.boundscheck(False)
@cython.wraparound(False) 
cdef V_BC ( long i, long  j,double [::1] CP,double [::1] CiP):
    cdef double P_SUM,iP_SUM
       
    if i == 1:
        P_SUM = CP[j-1]
        iP_SUM = CiP[j-1]
    else:
        P_SUM = CP[j-1]-CP[i-2]
        iP_SUM = CiP[j-1]-CiP[i-2]
    
    try:
        return (P_SUM*(iP_SUM/P_SUM)**2)
    except:
        return 0

@cython.boundscheck(False)
@cython.wraparound(False) 
cdef mean( long i, long  j,double [::1] CP,double [::1] Cip):
    cdef double P_SUM,iP_SUM
       
    if i == 1:
        P_SUM = CP[j-1]
        iP_SUM = Cip[j-1]
    else:
        P_SUM = CP[j-1]-CP[i-2]
        iP_SUM = Cip[j-1]-Cip[i-2]
    
    try:
        return iP_SUM/P_SUM
    except:
        return 0
    
@cython.boundscheck(False)
@cython.wraparound(False) 
cdef lookup(long col,long j,long row,double [:,::1] C,double [::1] CP,double [::1] CiP):
    return C[col+j,j] + V_BC(col+(j+1) , row+(j+1),CP,CiP) 

@cython.boundscheck(False)
@cython.wraparound(False) 
cdef SMAWK(long [::1] mri,long [::1] mci, long  j,double [:,::1] C,long [:,::1] D,long [::1]mposi,double [::1] CP,double [::1] CiP):
    cdef long [::1] aci
    cdef long [::1] bri

    if mci.shape[0] != mri.shape[0]:
        aci=REDUCE(mri,mci,j,C,CP,CiP)
    else:
        aci=mci

    if (mri.shape[0]==1 and aci.shape[0]==1):
        C[mri[0]+j+1,j+1]= lookup(aci[0],j,mri[0],C,CP,CiP)
        mposi[mri[0]]=D[mri[0]+j,j]=aci[0]

        return   

    bri = mri[1::2].copy()

    SMAWK(bri,aci,j,C,D,mposi,CP,CiP)
    MFILL(mri,aci,j,C,D,mposi,CP,CiP)

@cython.boundscheck(False)
@cython.wraparound(False) 
cdef REDUCE( long [::1] rows, long  [::1] cols, long  j,double [:,::1] C,double [::1] CP,double [::1] CiP):
    cdef long p = rows.shape[0]
    cdef long ncols = cols.shape[0]
    cdef long m = cols.shape[0]
    
    predd = np.arange(-1,m+1,dtype=np.int64)
    cdef long [::1] pred = predd
    valuee = np.full((m+1),-1,dtype=np.double)
    cdef double [::1] value = valuee

    cdef long a=2 
    cdef long b=1 
    
    rett = np.empty(p,dtype=np.int64)
    cdef long [::1] ret = rett
    cdef long lc = ncols+1
    
    
    value[1]=lookup(cols[0],j,rows[0],C,CP,CiP) 

    while m > p:
        if value[pred[a]] == -1:
            value[pred[a]] = (lookup(cols[pred[a]-1] ,j, rows[b-1],C,CP,CiP) if cols[pred[a]-1] <= rows[b-1] else 0)

        if value[pred[a]] >= (lookup(cols[a-1] ,j, rows[b-1],C,CP,CiP) if cols[a-1] <= rows[b-1] else 0) :
            if b < p :
                b = b+1
                value[a] = (lookup(cols[a-1] ,j, rows[b-1],C,CP,CiP) if cols[a-1] <= rows[b-1] else 0)
            else:
                pred[a+1] = pred[a]
                m = m-1

            a = a+1

        else:
            pred[a] = pred[pred[a]]
            m = m-1
            if b != 1:
                b = b-1
            else:
                a = a+1

    for i in range(p-1,-1,-1):
        ret[i]=cols[pred[lc]-1]
        lc=pred[lc]
    return ret

@cython.boundscheck(False)
@cython.wraparound(False) 
cdef MFILL( long [::1] ari, long  [::1] aci, long  j,double [:,::1] C,long [:,::1] D,long [::1]mposi,double [::1] CP,double [::1] CiP):
    cdef long m = ari.shape[0]
    cdef long n = aci.shape[0]
    cdef long ii = n-1
    cdef long start

    if (m % 2) == 0:
        start = m-2
    else:
        start = m-1

    cdef long s,e,ar,ac,i
    cdef double MAX,cc,vv,CURRENT_MAT
    for i in range(start,-1,-2):
        if (i==0):
            s=int(aci[i])
            e=int(mposi[ari[i+1]])

        elif (i==m-1):
            s=int(mposi[ari[i-1]])
            e=int(aci[n-1])

        else:
            s=int(mposi[ari[i-1]])
            e=int(mposi[ari[i+1]])

        if(e > ari[i]):
            e=int(ari[i])                     

        MAX = 0
        while True:
            ac = aci[ii]
            ar = ari[i]
            if (ac > ar):
                pass 
            else:
                CURRENT_MAT = lookup(ac,j,ar,C,CP,CiP)

                if (MAX < CURRENT_MAT):
                    MAX = CURRENT_MAT
                    mposi[ar]=ac
                    C[ar+j+1,j+1]=MAX
                    D[ar+j,j]=ac+j
            if(ac<=s or ii==-1):
                break
            ii-=1

@cython.boundscheck(False)
@cython.wraparound(False) 
cdef findBestK( long n, long  k,long [:,::1] D,double [::1] P,double [::1] CP,double [::1] Cip,double [::1] CF):
    cdef double E1 = 0
    cdef double mean1 = mean(1,n,CP,Cip)
    cdef long J,kk,j,a,K,i,t
    for J in range(1,n):
        E1 += (P[J])*(abs(J-mean1))

    cdef double bestAlpha = 0
    cdef long bestK = 0

    cdef double Dk,Ek,meanK,A,alpha
    cdef long [::1] T
    cdef long [::1] bestT
    
    for kk in range(k,1,-1):
        T = np.zeros(kk+2,dtype=np.int64)
        T[0] = 1
        T[kk+1] = n
        i = n
        for j in range(kk,0,-1):
            T[j] = D[i-1,j]
            i = int(D[i-1,j])
            if i < 1:
                i=1

        Dk = mean(T[kk],T[kk+1],CP,Cip)-mean(T[0],T[1],CP,Cip)
        Ek = 1

        for K in range(kk+1):
            meanK = mean(T[K],T[K+1],CP,Cip)
            for J in range(T[K],T[K+1]):
                Ek += (P[J])*(abs(J-meanK))

        A = 1
        for a in range (kk+1):
            t = T[a]
            A += P[t]

        A *= math.sqrt(kk)
        alpha=((((E1/Ek)*Dk)**2)/A)  

        if alpha > bestAlpha :
            bestK = kk
            bestT = T
            bestAlpha = alpha

    cdef double [::1] vol= np.empty(bestK)
    cdef double [::1] leng = np.empty(bestK)
    if bestK != 0:
        for i in range (1,bestK+1):
            vol[i-1]=(CF[bestT[i]]-CF[bestT[i-1]])
            leng[i-1]=(bestT[i]-bestT[i-1])
    else:
        bestT = np.empty(0,dtype=long)
    
    return bestT,bestK,vol,leng

@cython.boundscheck(False)
@cython.wraparound(False) 
def L(float [::1] F, long  k):
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import math
    import copy
    import time
    import os

    
    cdef long n = len(F)
    cdef double F_SUM = sum(F)
    if F_SUM == 0:
        return [],0,[],[]

    CC = np.zeros((n+1,k+2),dtype=np.float64)
    DD = np.zeros((n+1,k+2),dtype=np.int64)
    cdef double [:, ::1] C = CC
    cdef long[:, ::1] D = DD  
    cdef long sizeMAT 
    cdef long[::1] mposi 

    cdef double [::1] P = np.empty(n)
    cdef double [::1] CP = np.empty(n)
    cdef double [::1] CF = np.empty(n)
    cdef double [::1] CiP = np.empty(n)
    cdef double [::1] Cip = np.empty(n)
    cdef double PP,iP,ip
    cdef double firstValue = F[0]/F_SUM
    CF[0]=(F[0])
    P[0]=(firstValue)
    CP[0]=(firstValue)
    CiP[0]=(firstValue)
    Cip[0]=(firstValue)
    cdef long i,j,dl
    
    for i in range(1,n):
        CF[i]=(CF[i-1]+F[i])
        PP = F[i]/F_SUM
        iP = PP * i
        ip = PP * (i+1)
        P[i]=(PP)
        CP[i]=(CP[i-1]+PP)
        CiP[i]=(CiP[i-1]+iP)
        Cip[i]=(Cip[i-1]+ip)
    
    mminTj = np.zeros(k+2,dtype=np.int64)
    mmaxTj = np.zeros(k+2,dtype=np.int64)
    cdef long[::1] minTj = mminTj
    cdef long[::1] maxTj = mmaxTj

    for j in range(0,k+2):
        if j == k+1:
            minTj[j] = n
        else:
            minTj[j] = j

    for j in range(0,k+2):
        if j == 0:
            maxTj[j] = 0
        else:
            maxTj[j] = n-k+j-1    

    cdef long l,tj,ex=k-2
    cdef double v,f
    cdef long [::1] initial_rc 
    for j in range (0,k+1): 

        if j == 0:
            for tj in range(minTj[j+1],maxTj[j+1]+1+ex):
                C[tj,j+1]=V_BC(1,tj,CP,CiP)
                
        else: 
            if (j>=(3)):
                ex-=1
            sizeMAT = n-k+1+ex
            
            if (j != k):
                mposi = np.zeros(sizeMAT-1,dtype=np.int64)
                initial_rc = np.arange(0,sizeMAT-1,dtype=int)
                SMAWK(initial_rc,initial_rc,j,C,D,mposi,CP,CiP)
            else:
                dl = minTj[k]
                for l in range(minTj[k],maxTj[k]+1):
                    f = V_BC(l+1,minTj[k+1],CP,CiP)
                    v = f + C[l,k]
                    if (C[minTj[k+1],k+1] < v ):
                        C[minTj[k+1],k+1] = v
                        dl = l
                D[maxTj[k],k]=dl

    D[n,k+1] = n

    cdef long [::1] bestT
    cdef long bestK
    cdef double [::1] vol
    cdef double [::1] leng
    bestT,bestK,vol,leng = findBestK(n,k,D,P,CP,Cip,CF)
    return (bestT,bestK,vol,leng)    

In [6]:
def window(tup):
    start = tup[0]
    end = tup[1]
    froM = start
    winSize = 10000
    to = froM+winSize
    '''
    fname = str(str(chromo)+"-"+str(start)+"-"+str(end)+".csv")
    with open(fname, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["START", "END", "LENGTH", "VOLUME", "RANGE"])
    '''
    df = pd.DataFrame(columns=["CHR","START", "END", "LENGTH", "VOLUME", "RANGE"])
    while(froM < end):
        try:
            bestT=[]
            bestK=0
            sF=F[froM:to]
            if len(sF) != winSize:
                sF=np.append(sF,np.zeros(winSize - len(sF),dtype=sF.dtype))
            bestT,bestK,vol,leng=(L(sF,6))

            '''
            with open(fname, 'a', newline='') as file:
                writer = csv.writer(file)
                for i in range(bestK):
                    writer.writerow([froM+bestT[i],froM+bestT[i+1],leng[i],vol[i],str(str(froM)+':'+str(to))])
            '''
            if bestK != 0:
                for i in range(bestK):
                    df = df.append({"CHR":chromo,"START":froM+bestT[i],"END":froM+bestT[i+1], "LENGTH":leng[i], "VOLUME":vol[i], "RANGE":str(str(froM)+':'+str(to))},ignore_index=True)

            if bestK == 0:
                froM+=winSize
            else:
                froM+=bestT[bestK]
            to = froM+winSize

        except Exception as e:
            print("From:{}, To:{}, bestK={}".format(froM,to,bestK))
            print('from Window, ',e)
            froM+=winSize
            to=froM+winSize
            
    return df

In [7]:
%%time

path="/home/musab/chip-seq/GM12878-H3K27Ac_COPY/"
file = "wgEncodeBroadHistoneGm12878H3k27acStdSig.bigWig"
bw = pyBigWig.open(str(path+file))
full_result_list = []
for chromo in bw.chroms():

    total_size = bw.chroms()[chromo]
    F=bw.values(chromo,0,total_size,numpy=True)
    F[np.isnan(F)] = 0


    n = multiprocessing.cpu_count()
    frag = math.ceil(total_size/n)
    job = []
    st=0
    while (st<total_size):
        en =st+frag
        job.append((st,en))
        st = en+1

    pool = multiprocessing.Pool(processes=n)
    r = pool.map(window,job)
    
    result = pd.concat(r)
    result = result.reset_index(drop=True)
    full_result_list.extend(r)

    result.to_csv(str(path+"results/"+chromo+'.csv'), index=False)
    pool.close()
    pool.join()
    print("Chromo {} Done !".format(chromo))
    
full_result = pd.concat(full_result_list)
full_result = full_result.reset_index(drop=True)
full_result.to_csv(str(path+"results/"+'full.csv'), index=False)

Chromo chr1 Done !
Chromo chr10 Done !
Chromo chr11 Done !
Chromo chr12 Done !
Chromo chr13 Done !
Chromo chr14 Done !
Chromo chr15 Done !
Chromo chr16 Done !
Chromo chr17 Done !
Chromo chr18 Done !
Chromo chr19 Done !
Chromo chr2 Done !
Chromo chr20 Done !
Chromo chr21 Done !
Chromo chr22 Done !
Chromo chr3 Done !
Chromo chr4 Done !
Chromo chr5 Done !
Chromo chr6 Done !
Chromo chr7 Done !
Chromo chr8 Done !
Chromo chr9 Done !
Chromo chrX Done !
CPU times: user 46.2 s, sys: 21.6 s, total: 1min 7s
Wall time: 28min 31s


In [99]:
import os
import glob
import pandas as pd
import math
os.chdir("/home/musab/chip-seq/GM12878-H3K27Ac/results/")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_csv = combined_csv.sort_values('VOLUME', ascending=False)
#export to csv
# del(combined_csv['RANGE'])
# del(combined_csv['VOLUME'])
# del(combined_csv['LENGTH'])

#combined_csv.to_csv( "annotate.bed", header=False,index=False, sep='\t',encoding='utf-8-sig')

In [100]:
anno = combined_csv

In [101]:
del(anno['RANGE'])
del(anno['VOLUME'])
del(anno['LENGTH'])

In [102]:
length = anno.shape[0]

In [106]:
annotate = anno.head(math.ceil(40821*1.5))#math.ceil(length*0.01))

In [107]:
annotate.shape

(61232, 3)

In [108]:
annotate.to_csv( "anno_1.5.bed", header=False,index=False, sep='\t',encoding='utf-8-sig')

In [54]:
annotate.tail()

,CHR,START,END,LENGTH,VOLUME,RANGE
3175757,chr22,44464078,44464904,826.0,35059.440052,44461472:44471472
1113048,chr22,44464078,44464904,826.0,35059.440052,44461472:44471472
3175757,chr22,44464078,44464904,826.0,35059.440052,44461472:44471472
213617,chr16,11831052,11834851,3799.0,35057.840108,11807487:11837487
2790,chr16,11831052,11834851,3799.0,35057.840108,11807487:11837487


In [62]:
anno.tail()

,CHR,START,END,LENGTH,VOLUME,RANGE
2064581,chr1,2633747,2633748,1.0,1.0,2633746:2643746
1884,chrX,7623848,7623849,1.0,1.0,7623846:7653846
4133071,chrX,10737348,10737349,1.0,1.0,10737346:10747346
2623,chrX,10737347,10737348,1.0,1.0,10737346:10767346
1873,chr1,2633748,2633749,1.0,1.0,2633746:2643746
